In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained('ashok2216/gpt2-amazon-sentiment-classifier-V1.0')
model = AutoModelForTokenClassification.from_pretrained('ashok2216/gpt2-amazon-sentiment-classifier-V1.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.classifier = torch.nn.Linear(model.classifier.in_features, 5)

In [ ]:
for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
import pandas as pd

data = pd.DataFrame({
    "text": [
        "This product is amazing!",
        "I didn't like this product at all! it was a waste of money",
        "Decent quality, but could be better",
        "Best purchae ever! highly recommended!",
        "Best purchae ever! highly recommended"
    ],
    "label": [4, 0, 2, 3, 1]
})

In [ ]:
inputs = tokenizer(
    data['text'].tolist(),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

In [ ]:
targets = torch.tensor(data['label'].values)
targets

tensor([4, 0, 2, 3, 1])

In [ ]:
inputs

{'input_ids': tensor([[  101,  2023,  4031,  2003,  6429,   999,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1045,  2134,  1005,  1056,  2066,  2023,  4031,  2012,  2035,
           999,  2009,  2001,  1037,  5949,  1997,  2769,   102],
        [  101, 11519,  3737,  1010,  2021,  2071,  2022,  2488,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2190, 16405, 11140,  6679,  2412,   999,  3811,  6749,   999,
           102,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2190, 16405, 11140,  6679,  2412,   999,  3811,  6749,   102,
             0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0

In [ ]:
outputs = model(**inputs, labels=targets)

ValueError: Expected input batch_size (225) to match target batch_size (5).

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch.nn as nn

# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('ashok2216/gpt2-amazon-sentiment-classifier-V1.0')
model = AutoModelForSequenceClassification.from_pretrained('ashok2216/gpt2-amazon-sentiment-classifier-V1.0')

# Configure model for 5 classes and single-label classification
model.config.num_labels = 5
model.config.problem_type = "single_label_classification"  # Ensure correct problem type

# Replace classifier with a new 5-class output layer
in_features = model.classifier.in_features
model.classifier = torch.nn.Linear(in_features, 5)

# Freeze all layers except the classifier
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

# Sample data with ratings 1-5
data = pd.DataFrame({
    "text": [
        "This product is amazing!",
        "I didn't like this product at all! it was a waste of money",
        "Decent quality, but could be better",
        "Best purchase ever! highly recommended!",
        "Not worth the price, but okay"
    ],
    "label": [5, 1, 3, 5, 2]
})

# Tokenize the inputs
inputs = tokenizer(
    data['text'].tolist(),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

# Adjust targets to zero-based labels
targets = torch.tensor([label - 1 for label in data['label']])

# Forward pass
outputs = model(**inputs)

# Extract logits and verify shape
logits = outputs.logits
print(f"logits shape: {logits.shape}")  # Should be (batch_size, 5)

# Manually calculate loss
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(logits, targets)
print(f"Initial Loss: {loss.item()}")

# Fine-tuning the classifier layer
loss.backward()
optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=1e-4)
optimizer.step()

logits shape: torch.Size([5, 5])
Initial Loss: 1.6115977764129639


In [ ]:
# Sample test data with new reviews to predict ratings
test_data = pd.DataFrame({
    "text": [
        "Absolutely love this product, exceeded my expectations!",
        "Terrible quality, would not recommend to anyone.",
        "It’s okay, does the job but nothing special.",
        "Excellent value for money, very satisfied.",
        "Not great, but not the worst either. Just average."
    ]
})

# Tokenize the test inputs
test_inputs = tokenizer(
    test_data['text'].tolist(),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

# Forward pass for predictions
with torch.no_grad():
    test_outputs = model(**test_inputs)
    test_logits = test_outputs.logits

# Convert logits to probabilities and get the predicted class
probabilities = torch.nn.functional.softmax(test_logits, dim=1)
predictions = torch.argmax(probabilities, dim=1) + 1  # Convert to 1-5 scale

# Display the results
for i, review in enumerate(test_data['text']):
    print(f"Review: {review}")
    print(f"Predicted Rating: {predictions[i].item()}")
    print(f"Probabilities: {probabilities[i].tolist()}")
    print("\n" + "-"*50 + "\n")

Review: Absolutely love this product, exceeded my expectations!
Predicted Rating: 2
Probabilities: [0.1660737693309784, 0.2757783830165863, 0.14232151210308075, 0.22083310782909393, 0.1949932724237442]

--------------------------------------------------

Review: Terrible quality, would not recommend to anyone.
Predicted Rating: 5
Probabilities: [0.23109613358974457, 0.15369099378585815, 0.15330205857753754, 0.21897432208061218, 0.24293649196624756]

--------------------------------------------------

Review: It’s okay, does the job but nothing special.
Predicted Rating: 5
Probabilities: [0.22556540369987488, 0.19455011188983917, 0.16045208275318146, 0.1852637082338333, 0.2341686338186264]

--------------------------------------------------

Review: Excellent value for money, very satisfied.
Predicted Rating: 2
Probabilities: [0.16764771938323975, 0.2692895233631134, 0.1375901848077774, 0.21660107374191284, 0.20887145400047302]

--------------------------------------------------

Review